In [1]:
# import the necessary packages
import numpy as np
import imutils
import time
from scipy import spatial
import cv2

In [2]:
list_of_vehicles = ["truck"]

In [3]:
LABELS = open('C:/vani_daivajna/transurban/src_code/bin/coco.names').read().strip().split('\n') 
weightsPath = 'C:/vani_daivajna/transurban/src_code/bin/yolov4.weights'
configPath = 'C:/vani_daivajna/transurban/src_code/bin/yolo4.cfg'



In [4]:
inputVideoPath="C:/vani_daivajna/transurban/input_video/clip5.mp4"
#inputVideoPath='op4.avi'
OUTPUT_FILE='C:/vani_daivajna/transurban/output_video/output_clip5.avi'

In [5]:
probability_minimum = 0.85
threshold = 0.2
FRAMES_BEFORE_CURRENT = 1000
inputWidth, inputHeight = 416, 416

In [6]:
def displayVehicleCount(frame, vehicle_count):
    #print("frame is "frame)
   #print("count is",vehicle_count)
    cv2.putText(
        frame, #Image
        
        'Trucks count ' + str(vehicle_count), #Label
        (20, 20), #Position
        cv2.FONT_HERSHEY_SIMPLEX, #Font
        0.8, #Size
        (0, 0, 0xFF), #Color
        2, #Thickness
        cv2.FONT_HERSHEY_COMPLEX_SMALL,
        )

In [7]:
#Determining if the box-mid point cross the line or are within the range of 5 units
#this will help if it is same object or not 
def boxAndLineOverlap(x_mid_point, y_mid_point, line_coordinates):
    x1_line, y1_line, x2_line, y2_line = line_coordinates #Unpacking

    if (x_mid_point >= x1_line and x_mid_point <= x2_line+(range(2,50))) and\
        (y_mid_point >= y1_line and y_mid_point <= y2_line+(range(2,50))):
        return True
    return False
    

In [8]:
#Displaying the FPS of the detected video
def displayFPS(start_time, num_frames):
    current_time = int(time.time())
    
    if(current_time > start_time):
        os.system('clear') # Equivalent of CTRL+L on the terminal
        #print("FPS:", num_frames)
        num_frames = 0
        start_time = current_time
    return start_time, num_frames

In [9]:
def drawDetectionBoxes(idxs, boxes, classIDs, confidences, frame):
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indices we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # draw a bounding box rectangle and label on the frame
            color = [0,255,0]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],
                confidences[i])
            cv2.putText(frame, text, (x, y - 5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            #Draw a green dot in the middle of the box
            cv2.circle(frame, (x + (w//2), y+ (h//2)), 2, (0, 0xFF, 0), thickness=2)

In [10]:
def initializeVideoWriter(video_width, video_height, videoStream):
    # Getting the fps of the source video
    sourceVideofps = videoStream.get(cv2.CAP_PROP_FPS)
    # initialize our video writer
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    return cv2.VideoWriter(OUTPUT_FILE, fourcc, sourceVideofps,(video_width, video_height), True)

In [11]:
#Identifying if the current box was present in the previous frames
def boxInPreviousFrames(previous_frame_detections, current_box, current_detections):
    centerX, centerY, width, height = current_box
    dist = np.inf #Initializing the minimum distance
    # Iterating through all the k-dimensional trees
    for i in range(FRAMES_BEFORE_CURRENT):
        coordinate_list = list(previous_frame_detections[i].keys())
        if len(coordinate_list) == 0: # When there are no detections in the previous frame
            continue
        # Finding the distance to the closest point and the index
        temp_dist, index = spatial.KDTree(coordinate_list).query([(centerX, centerY)])
       
        if (temp_dist < dist):
            dist = temp_dist
            frame_num = i
            coord = coordinate_list[index[0]]
    #print("-----------")
    #print(coord)
    #print(dist)
    #print(width/2)
    #print(height/2)
   
    if (dist > (max(width, height))/2):
        return False

    #if (dist >150):
        #print(dist)
        #return False
    #print("-----------------------")
    #print(dist)
    # Keeping the vehicle ID constant
    current_detections[(centerX, centerY)] = previous_frame_detections[frame_num][coord]
    return True


In [12]:
def count_vehicles(idxs, boxes, classIDs, vehicle_count, previous_frame_detections, frame):
    current_detections = {}
    # ensure at least one detection exists
    if len(idxs) > 0:
        # loop over the indices we are keeping
        for i in idxs.flatten():
            # extract the bounding box coordinates
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            centerX = x + (w//2)
            centerY = y+ (h//2)

            # When the detection is in the list of vehicles, AND
            # it crosses the line AND
            # the ID of the detection is not present in the vehicles
            if (LABELS[classIDs[i]] in list_of_vehicles):
                current_detections[(centerX, centerY)] = vehicle_count 
                if (not boxInPreviousFrames(previous_frame_detections, (centerX, centerY, w, h), current_detections)):
                    vehicle_count += 1
                    # vehicle_crossed_line_flag += True
                    # else: #ID assigning
                    #Add the current detection mid-point of box to the list of detected items
                    # Get the ID corresponding to the current detection

                ID = current_detections.get((centerX, centerY))
                # If there are two detections having the same ID due to being too close, 
                # then assign a new ID to current detection.
                if (list(current_detections.values()).count(ID) > 1):
                    current_detections[(centerX, centerY)] = vehicle_count
                    vehicle_count += 1 

#Display the ID at the center of the box
                cv2.putText(frame, str(ID), (centerX, centerY),\
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0,0,255], 2)

    return vehicle_count, current_detections

In [13]:
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

In [14]:
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [15]:
videoStream = cv2.VideoCapture(inputVideoPath)
video_width = int(videoStream.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(videoStream.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [16]:
# Specifying coordinates for a default line 
x1_line = 0
y1_line = video_height//2
x2_line = video_width
y2_line = video_height//2

In [17]:
#Initialization
import os
previous_frame_detections = [{(0,0):0} for i in range(FRAMES_BEFORE_CURRENT)]
#previous_frame_detections = [spatial.KDTree([(0,0)])]*FRAMES_BEFORE_CURRENT # Initializing all trees
num_frames, vehicle_count = 0, 0
writer = initializeVideoWriter(video_width, video_height, videoStream)
start_time = int(time.time())
cnt=0
# loop over frames from the video file stream
while True:
    cnt+=1
    #print("frame is ",cnt)
    num_frames+= 1
    #print("current frame is:", num_frames)
    # Initialization for each iteration
    boxes, confidences, classIDs = [], [], [] 
    vehicle_crossed_line_flag = False 

    #Calculating fps each second
    start_time, num_frames = displayFPS(start_time, num_frames)
    # read the next frame from the file
    (grabbed, frame) = videoStream.read()

    # if the frame was not grabbed, then we have reached the end of the stream
    if not grabbed:
        break


    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (inputWidth, inputHeight),swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    for output in layerOutputs:
        for i, detection in enumerate(output):
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]


            if confidence > probability_minimum:
                if (classID==7):
                    box = detection[0:4] * np.array([video_width, video_height, video_width, video_height])
                    
                    
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)
    idxs = cv2.dnn.NMSBoxes(boxes, confidences,probability_minimum,threshold)
    drawDetectionBoxes(idxs, boxes, classIDs, confidences, frame)
    vehicle_count, current_detections = count_vehicles(idxs, boxes, classIDs, vehicle_count, previous_frame_detections, frame)
    #print(vehicle_count, current_detections)
    displayVehicleCount(frame, vehicle_count)

    # write the output frame to diskd
    writer.write(frame)

    cv2.imshow('Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break	

    # Updating with the current frame detections
    previous_frame_detections.pop(0) #Removing the first frame from the list
    # previous_frame_detections.append(spatial.KDTree(current_detections))
    previous_frame_detections.append(current_detections)

        # release the file pointers
print(vehicle_count)
print("[INFO] cleaning up...")
writer.release()
videoStream.release()

5
[INFO] cleaning up...


In [18]:
import cv2
import os
  
# Read the video from specified path
cam = cv2.VideoCapture(OUTPUT_FILE)
  
op=str(OUTPUT_FILE).strip().split('/')[-1]
op=str(op).strip().split('.') 
op="frame_by_frame_"+op[0]
print(op)
try:
      
    # creating a folder named data
    if not os.path.exists("C:/vani_daivajna/transurban/truck_count_frame_by_frame/"+op):
        os.makedirs("C:/vani_daivajna/transurban/truck_count_frame_by_frame/"+op)
  
# if not created then raise error
except OSError:
    print ('Error: Creating directory of data')
  
# frame
currentframe = 0
  
while(True):
      
    # reading from frame
    ret,frame = cam.read()
  
    if ret:
        # if video is still left continue creating images
        name = "C:/vani_daivajna/transurban/truck_count_frame_by_frame/"+op+'/frame' + str(currentframe) + '.jpg'
        print ('Creating...' + name)
  
        # writing the extracted images
        cv2.imwrite(name, frame)
  
        # increasing counter so that it will
        # show how many frames are created
        currentframe += 1
    else:
        break
  
# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()

frame_by_frame_output_clip5
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame0.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame1.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame2.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame3.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame4.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame5.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame6.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame7.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame8.jpg
Creating...C:/vani_daivajna

Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame80.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame81.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame82.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame83.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame84.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame85.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame86.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame87.jpg
Creating...C:/vani_daivajna/transurban/truck_count_frame_by_frame/frame_by_frame_output_clip5/frame88.jpg
Creating...C:/vani_daivajna/transurban/truck_c

In [ ]:
#print(videoStream.get(cv2.CAP_PROP_POS_MSEC))


In [ ]:
import pandas as pd
c_df=pd.read_excel("vehicle_count.xlsx")
df = pd.DataFrame(columns=['path_of_video','height','width','Vehicle_tpe','Count of vehicle'])

In [ ]:
p=[]
p.append(str(inputVideoPath))
df["path_of_video"]=p
#seconds = int(frames / fps)
vs = cv2.VideoCapture(inputVideoPath)
vs.set(cv2.CAP_PROP_POS_AVI_RATIO,1)

#l=[]
#l.append((vs.get(cv2.CAP_PROP_POS_MSEC)))
#df["video duration"]=l
h=[]
h.append(video_height)
w=[]
w.append(video_width)
df["height"]=h
df["width"]=w
df["Vehicle_tpe"]='truck'
c=[]
c.append(vehicle_count)
df["Count of vehicle"]=c

In [ ]:
df.head()

In [ ]:
c_df=c_df.append(df)

In [ ]:
c_df.head()

In [ ]:
c_df.to_excel("vehicle_count.xlsx",index=False)